In [1]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import standard_gates
from qiskit.quantum_info import SparsePauliOp
from qiskit.providers.fake_provider import FakeLima
from scripts.utils import create_estimator_meas_data
import numpy as np
import random
import pickle

def create_random_circuit(n_qubits, max_two_qubit_depth):
    qc = QuantumCircuit(n_qubits)
    
    single_qubit_gates = [
        # (Gate class, number of qubits, number of parameters)
        (standard_gates.IGate, 1, 0),
        (standard_gates.SXGate, 1, 0),
        (standard_gates.XGate, 1, 0),
        (standard_gates.RZGate, 1, 1),
        (standard_gates.RGate, 1, 2),
        (standard_gates.HGate, 1, 0),
        (standard_gates.PhaseGate, 1, 1),
        (standard_gates.RXGate, 1, 1),
        (standard_gates.RYGate, 1, 1),
        (standard_gates.SGate, 1, 0),
        (standard_gates.SdgGate, 1, 0),
        (standard_gates.SXdgGate, 1, 0),
        (standard_gates.TGate, 1, 0),
        (standard_gates.TdgGate, 1, 0),
        (standard_gates.UGate, 1, 3),
        (standard_gates.U1Gate, 1, 1),
        (standard_gates.U2Gate, 1, 2),
        (standard_gates.U3Gate, 1, 3),
        (standard_gates.YGate, 1, 0),
        (standard_gates.ZGate, 1, 0),
    ]

    two_qubit_gates = [
        (standard_gates.CXGate, 2, 0),
        (standard_gates.DCXGate, 2, 0),
        (standard_gates.CHGate, 2, 0),
        (standard_gates.CPhaseGate, 2, 1),
        (standard_gates.CRXGate, 2, 1),
        (standard_gates.CRYGate, 2, 1),
        (standard_gates.CRZGate, 2, 1),
        (standard_gates.CSXGate, 2, 0),
        (standard_gates.CUGate, 2, 4),
        (standard_gates.CU1Gate, 2, 1),
        (standard_gates.CU3Gate, 2, 3),
        (standard_gates.CYGate, 2, 0),
        (standard_gates.CZGate, 2, 0),
        (standard_gates.RXXGate, 2, 1),
        (standard_gates.RYYGate, 2, 1),
        (standard_gates.RZZGate, 2, 1),
        (standard_gates.RZXGate, 2, 1),
        (standard_gates.XXMinusYYGate, 2, 2),
        (standard_gates.XXPlusYYGate, 2, 2),
        (standard_gates.ECRGate, 2, 0),
        (standard_gates.CSGate, 2, 0),
        (standard_gates.CSdgGate, 2, 0),
        (standard_gates.SwapGate, 2, 0),
        (standard_gates.iSwapGate, 2, 0),
    ]
    
    for depth in range(max_two_qubit_depth):
        # Always add at least one two-qubit gate
        qubit1, qubit2 = random.sample(range(n_qubits), 2)
        gate_class, _, num_params = random.choice(two_qubit_gates)
        params = [random.uniform(0, 2*np.pi) for _ in range(num_params)]
        gate = gate_class(*params) if num_params > 0 else gate_class()
        qc.append(gate, [qubit1, qubit2])
        
        used_qubits = set([qubit1, qubit2])
        
        # Possibly add more two-qubit gates
        for _ in range((n_qubits // 2) - 1):  # -1 because we've already added one
            available_qubits = list(set(range(n_qubits)) - used_qubits)
            if len(available_qubits) >= 2:
                qubit1, qubit2 = random.sample(available_qubits, 2)
                gate_class, _, num_params = random.choice(two_qubit_gates)
                params = [random.uniform(0, 2*np.pi) for _ in range(num_params)]
                gate = gate_class(*params) if num_params > 0 else gate_class()
                qc.append(gate, [qubit1, qubit2])
                used_qubits.update([qubit1, qubit2])
        
        # Randomly add single-qubit gates
        for qubit in range(n_qubits):
            if random.random() < 0.5:  # 50% chance to add a single-qubit gate
                gate_class, _, num_params = random.choice(single_qubit_gates)
                params = [random.uniform(0, 2*np.pi) for _ in range(num_params)]
                gate = gate_class(*params) if num_params > 0 else gate_class()
                qc.append(gate, [qubit])
    
    return qc

def check_two_qubit_depth(circuit):
    two_qubit_depth = 0
    current_layer = set()
    
    for instruction in circuit.data:
        if len(instruction.qubits) == 2:  # It's a two-qubit gate
            qubit1, qubit2 = instruction.qubits
            if qubit1 in current_layer or qubit2 in current_layer:
                # This gate can't be applied in parallel with the current layer
                two_qubit_depth += 1
                current_layer = set([qubit1, qubit2])
            else:
                # This gate can be applied in parallel with the current layer
                current_layer.update([qubit1, qubit2])
        elif len(current_layer) > 0:
            # It's a single-qubit gate, but we've seen two-qubit gates in this layer
            two_qubit_depth += 1
            current_layer = set()
    
    # If there are gates in the last layer
    if len(current_layer) > 0:
        two_qubit_depth += 1
    
    return two_qubit_depth

def create_data(n_qubits: int = 4, 
                min_two_qubit_depth: int = 30,
                max_two_qubit_depth: int = 40, 
                n_train: int = 500,
                n_test: int = 200
                ):

    backend = FakeLima()
    
    train_data = {depth: [] for depth in range(1, max_two_qubit_depth + 1)}
    test_data = {depth: [] for depth in range(1, max_two_qubit_depth + 1)}

    observables = SparsePauliOp.from_list([('Z' + 'I'*(n_qubits-1), 1),
                                          ('I' + 'Z' + 'I'*(n_qubits-2), 1),
                                          ('I'*2 + 'Z' + 'I'*(n_qubits-3), 1),
                                          ('I'*(n_qubits-1) + 'Z', 1)])

    for depth in range(min_two_qubit_depth, max_two_qubit_depth + 1):
        for _ in range(n_train):
            qc = create_random_circuit(n_qubits, depth)
            actual_depth = check_two_qubit_depth(qc)
            if actual_depth != depth:
                print(f"Warning: Intended depth {depth}, actual depth {actual_depth}")
            ideal_exp_vals = []
            noisy_exp_vals = []
            for observable in observables:
                ideal_exp_val, noisy_exp_val = create_estimator_meas_data(
                    backend=backend, circuit=qc, observable=observable
                )
                ideal_exp_vals.append(ideal_exp_val)
                noisy_exp_vals.append(noisy_exp_val)
            train_data[depth].append((qc, observables, ideal_exp_vals, noisy_exp_vals))
        
        for _ in range(n_test):
            qc = create_random_circuit(n_qubits, depth)
            actual_depth = check_two_qubit_depth(qc)
            if actual_depth != depth:
                print(f"Warning: Intended depth {depth}, actual depth {actual_depth}")
            ideal_exp_vals = []
            noisy_exp_vals = []
            for observable in observables:
                ideal_exp_val, noisy_exp_val = create_estimator_meas_data(
                    backend=backend, circuit=qc, observable=observable
                )
                ideal_exp_vals.append(ideal_exp_val)
                noisy_exp_vals.append(noisy_exp_val)
            test_data[depth].append((qc, observables, ideal_exp_vals, noisy_exp_vals))

    return train_data, test_data

def save_data(train_data, test_data, file_name):
    data = {
        "train_data": train_data,
        "test_data": test_data,
    }
    with open(file_name, 'wb') as f:
        pickle.dump(data, f)

def create_and_save_data(file_name: str):
    train_data, test_data = create_data()
    save_data(train_data, test_data, file_name)

def load_data(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data["train_data"], data["test_data"]

In [2]:
# Create and save the data
create_and_save_data("random_circ_paper_experiment_30_40.pkl")

# To load the data later
train_data, test_data = load_data("random_circ_paper_experiment_18_30.pkl")